[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/search/semantic-search/yt-search/01-yt-search.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/search/semantic-search/yt-search/01-yt-search.ipynb)

Install prerequisite packages.

In [ ]:
!pip install -U datasets sentence-transformers pinecone-client tqdm

# YouTube Indexing and Queries

In this notebook we will work through an example of indexing and querying the YouTube video transcriptions data. We start by loading the dataset.

In [ ]:
from datasets import load_dataset

ytt = load_dataset(
    "pinecone/yt-transcriptions",
    split="train",
    revision="926a45"
)
ytt

Using custom data configuration pinecone--yt-transcriptions-b2ddba2bc158a89e
Reusing dataset json (/Users/jamesbriggs/.cache/huggingface/datasets/json/pinecone--yt-transcriptions-b2ddba2bc158a89e/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b)


Dataset({
    features: ['video_id', 'text', 'start_second', 'end_second', 'url', 'title', 'thumbnail'],
    num_rows: 11298
})

Each sample includes video-level information (ID, title, url and thumbnail) and snippet-level information (text, start_second, end_second).

In [ ]:
for x in ytt:
    print(x)
    break

{'video_id': 'ZPewmEu7644', 'text': " hi this is Jeff Dean welcome to applications of deep neural networks of Washington University in this video we're going to look at how we can use ganz to generate additional training data for the latest on my a I course and projects click subscribe in the bell next to it to be notified of every new video Dan's have a wide array of uses beyond just the face generation that you", 'start_second': 0, 'end_second': 20, 'url': 'https://www.youtube.com/watch?v=ZPewmEu7644&t=0s', 'title': 'GANS for Semi-Supervised Learning in Keras (7.4)', 'thumbnail': 'https://i.ytimg.com/vi/ZPewmEu7644/maxresdefault.jpg'}


# Uploading Documents to Pinecone Index

The next step is indexing this dataset in Pinecone, for this we need a sentence transformer model (to encode the text into embeddings), and a Pinecone index.

We will initialize the sentence transformer first.

In [ ]:
from sentence_transformers import SentenceTransformer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

retriever = SentenceTransformer(
    'flax-sentence-embeddings/all_datasets_v3_mpnet-base'
)
retriever.to(device)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

We can see the embedding dimension of `768` above, we will need this when creating our Pinecone index.

In [ ]:
embed_dim = retriever.get_sentence_embedding_dimension()
embed_dim

768

Now we can initialize our index.

In [ ]:
import pinecone

# get api key from app.pinecone.io
pinecone.init(
    api_key="<<YOUR_API_KEY>>",
    environment="us-west1-gcp"
)

# create index
pinecone.create_index(
    "youtube-search",
    dimension=embed_dim,
    metric="cosine"
)

# connect to new index
index = pinecone.Index("youtube-search")

We will index our data in batches of `64`, the data we insert into our index will contain records (eg *documents*) containing a unique document/snippet ID, embedding, and metadata in the format:

```json
{
    'doc-id',
    [0.0, 0.3, 0.1, ...],
    {'title': '???', 'start_seconds': 12, ...}
}
```

In [ ]:
from tqdm.auto import tqdm

docs = []  # this will store IDs, embeddings, and metadata

batch_size = 64

for i in tqdm(range(0, len(ytt), batch_size)):
    i_end = min(i+batch_size, len(ytt))
    # extract batch from YT transactions data
    batch = ytt[i:i_end]
    # encode batch of text
    embeds = retriever.encode(batch['text']).tolist()
    # each snippet needs a unique ID
    # we will merge video ID and start_seconds for this
    ids = [f"{x[0]}-{x[1]}" for x in zip(batch['video_id'], batch['start_second'])]
    # create metadata records
    meta = [{
        'video_id': x[0],
        'title': x[1],
        'text': x[2],
        'start_second': x[3],
        'end_second': x[4],
        'url': x[5],
        'thumbnail': x[6]
    } for x in zip(
        batch['video_id'],
        batch['title'],
        batch['text'],
        batch['start_second'],
        batch['end_second'],
        batch['url'],
        batch['thumbnail']
    )]
    # create list of (IDs, vectors, metadata) to upsert
    to_upsert = list(zip(ids, embeds, meta))
    # add to pinecone
    index.upsert(vectors=to_upsert)

  0%|          | 0/177 [00:00<?, ?it/s]

In [ ]:
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.01,
 'namespaces': {'': {'vector_count': 11298}}}

# Querying

When query we encode our text with the same retriever model and pass it to the Pinecone `query` endpoint.

In [ ]:
query = "What is deep learning?"

xq = retriever.encode([query]).tolist()

In [ ]:
xc = index.query(xq, top_k=5,
                 include_metadata=True)
for context in xc['results'][0]['matches']:
    print(context['metadata']['text'], end="\n---\n")

 terms of optimization but what's the algorithm for updating the parameters or updating whatever the state of the network is and then the the last part is the the data set like how do you actually represent the world as it comes into your machine learning system so I think of deep learning as telling us something about what does the model look like and basically to qualify as deep I
---
 any theoretical components any theoretical things that you need to understand about deep learning can be sick later for that link again just watched the word doc file again in that I mentioned the link also the second channel is my channel because deep learning might be complete deep learning playlist that I have created is completely in order okay to the other
---
 under a rock for the last few years you have heard of the deep networks and how they have revolutionised computer vision and kind of the standard classic way of doing this is it's basically a classic supervised learning problem you are givi

# Delete the Index

If you're done with the index, we delete it to save resources.

In [ ]:
pinecone.delete_index("youtube-search")

---